# Import Modules

In [ ]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt

# Defining global variables

In [ ]:
# V1 Base URL
global_url_v1_modules = "https://registry.terraform.io/v1/modules"
global_url_v1_provider = "https://registry.terraform.io/v1/providers"

# V2 Base URL
global_url_v2_modules = "https://registry.terraform.io/v2/modules"
global_url_v2_provider = "https://registry.terraform.io/v2/providers"

# GitHub Repository URL for Providers
global_url_github_provider_aws = "https://github.com/hashicorp/terraform-provider-aws"
global_url_github_provider_azurerm = "https://github.com/hashicorp/terraform-provider-azurerm"
global_url_github_provider_google = "https://github.com/hashicorp/terraform-provider-google"

# Defining functions

In [ ]:
# Get JSON data from URL
def get_json_data(url):
    response = requests.get(url)
    data = json.loads(response.text)
    return data

In [ ]:
def get_pagination_v1(url):
    data = get_json_data(url)
    return data['meta']

In [ ]:
def get_pagination_v1_from_json(data):
    return data['meta']

In [ ]:
# Returns from the given URL the pagination data from JSON
def get_pagination_v2(url):
    data = get_json_data(url)
    return data['meta']['pagination']

In [ ]:
def next_offset_from_url(url):
    meta_data = get_pagination_v1(url)
    return meta_data['next_offset']

In [ ]:
def next_offset_from_json(data):
    return data['next_offset']

In [ ]:
# NOTE: Please use number+=15
def get_page_v1(url, number):
    url = url + "?offset=" + str(number)
    response = requests.get(url)
    data = json.loads(response.text)
    return data

get_page_v1(global_url_v1_modules, 0)

In [ ]:
def v1_get_modules_from_provider(url, provider):
    local_list_modules = []
    # set start URL
    url = url + "?provider=" + str(provider)
    while True:
        # get JSON data from URL
        data = get_json_data(url)

        # get meta data header from current URL
        page_meta_data = get_pagination_v1_from_json(data)

        # if there is a next offset, get the modules
        if "next_offset" in page_meta_data:
            # get the next offset from loaded meta data
            next_offset = next_offset_from_json(page_meta_data)

            # append the modules to the local list
            for module in data['modules']:
                local_list_modules.append(module)

            # update the url with the next offset
            url = global_url_v1_modules + "?provider=" + str(provider) + "&offset=" + str(next_offset)
        # if there is no next offset, break the loop
        else:
            # attend the displayed modules in the last page too
            for module in data['modules']:
                local_list_modules.append(module)

            break
            
    return local_list_modules   

In [ ]:
def get_modules_v2(url, pages):
    local_list_modules = []
    for i in range(pages):
        url_page = url + "?page%5Bnumber%5D=" + str(i+1) + "&page%5Bsize%5D=15"
        data = get_json_data(url_page)
        print(i)
        # convert json data to list
        for item in data['data']:
            local_list_modules.append(item)

    return local_list_modules

# Providers

In [ ]:
print(get_pagination_v2(global_url_v2_provider))

# Modules

In [ ]:
len(v1_get_modules_from_provider(global_url_v1_modules, "google"))